<a href="https://colab.research.google.com/github/studio-ousia/luke/blob/master/notebooks/huggingface_open_entity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reproducing experimental results of LUKE on Open Entity Using Hugging Face Transformers

This notebook shows how to reproduce the state-of-the-art results on the [Open Entity entity typing dataset](https://www.cs.utexas.edu/~eunsol/html_pages/open_entity.html) reported in [this paper](https://arxiv.org/abs/2010.01057) using the Trasnsformers library and the [fine-tuned model checkpoint](https://huggingface.co/studio-ousia/luke-large-finetuned-open-entity) available on the Model Hub.
The source code used in the experiments is also available [here](https://github.com/studio-ousia/luke/tree/master/examples/entity_typing).

In [1]:
!pip install git+https://github.com/huggingface/transformers.git@refs/pull/11223/head

  Cloning https://github.com/huggingface/transformers.git (to revision refs/pull/11223/head) to /tmp/pip-req-build-hideogzv
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-hideogzv
  Running command git fetch -q https://github.com/huggingface/transformers.git refs/pull/11223/head
  Running command git checkout -q 9f727b5d8640babc0e6029da682516ecbf88844d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3MB 20.0MB/s 
     |████████████████████████████████| 901kB 51.6MB/s 
  Created wheel for transformers: filename=transformers-4.6.0.dev0-cp37-none-any.whl size=2151927 sha256=dee4f1034fc58d148299e3c0e79c9f537cf9c9f06f455b3fa64473d0477af192
  Stored in directory: /tmp/pip-ephem-wheel-cache-8j2dlo39/wheels/17/a9/0b/d1b18d07749e24c87f3e9a2045220a1cfa7d31ae75994e3270
Successfully built transformers


In [2]:
import json
import torch
from tqdm import trange
from transformers import LukeTokenizer, LukeForEntityClassification

## Loading the dataset

The dataset is downloaded from the link mentioned in [this GitHub repository](https://github.com/thunlp/ERNIE). The test.json file is placed in the current directory and loaded using the `load_examples` function.

In [3]:
!gdown --id 1HlWw7Q6-dFSm9jNSCh4VaBf1PlGqt9im
!tar xzf /content/data.tar.gz

# Place test.json on the working directory
!cp data/OpenEntity/test.json .

Downloading...
From: https://drive.google.com/uc?id=1HlWw7Q6-dFSm9jNSCh4VaBf1PlGqt9im
To: /content/data.tar.gz
322MB [00:03, 87.5MB/s]


In [4]:
def load_examples(dataset_file):
    with open(dataset_file, "r") as f:
        data = json.load(f)

    examples = []
    for item in data:
        examples.append(dict(
            text=item["sent"],
            entity_spans=[(item["start"], item["end"])],
            label=item["labels"]
        ))

    return examples

In [5]:
test_examples = load_examples("test.json")

## Loading the fine-tuned model and tokenizer

We construct the model and tokenizer using the [fine-tuned model checkpoint](https://huggingface.co/studio-ousia/luke-large-finetuned-open-entity).

In [6]:
# Load the model checkpoint
model = LukeForEntityClassification.from_pretrained("studio-ousia/luke-large-finetuned-open-entity")
model.eval()
model.to("cuda")

# Load the tokenizer
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-large-finetuned-open-entity")

Some weights of the model checkpoint at studio-ousia/luke-large-finetuned-open-entity were not used when initializing LukeForEntityClassification: ['luke.embeddings.position_ids']
- This IS expected if you are initializing LukeForEntityClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LukeForEntityClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Measuring performance

We classify entity mentions in the test set and measure the performance of the model.
The performance reported in the [original paper](https://arxiv.org/abs/2010.01057) is successfully reproduced.

In [7]:
batch_size = 128

num_predicted = 0
num_gold = 0
num_correct = 0

all_predictions = []
all_labels = []

for batch_start_idx in trange(0, len(test_examples), batch_size):
    batch_examples = test_examples[batch_start_idx:batch_start_idx + batch_size]
    texts = [example["text"] for example in batch_examples]
    entity_spans = [example["entity_spans"] for example in batch_examples]
    gold_labels = [example["label"] for example in batch_examples]

    inputs = tokenizer(texts, entity_spans=entity_spans, return_tensors="pt", padding=True)
    inputs = inputs.to("cuda")
    with torch.no_grad():
        outputs = model(**inputs)

    num_gold += sum(len(l) for l in gold_labels)
    for logits, labels in zip(outputs.logits, gold_labels):
        for index, logit in enumerate(logits):
            if logit > 0:
                num_predicted += 1
                predicted_label = model.config.id2label[index]
                if predicted_label in labels:
                    num_correct += 1

precision = num_correct / num_predicted
recall = num_correct / num_gold
f1 = 2 * precision * recall / (precision + recall)

print(f"\n\nprecision: {precision} recall: {recall} f1: {f1}")

100%|██████████| 16/16 [00:34<00:00,  2.14s/it]



precision: 0.7980295566502463 recall: 0.7657563025210085 f1: 0.781559903511123


## Detecting types of entities in a text

Finally, we detect types of entities in a text using the [fine-tuned model](https://huggingface.co/studio-ousia/luke-large-finetuned-open-entity).

In [13]:
text = "Beyoncé lives in Los Angeles."
entity_spans = [(0, 7)]  # character-based entity span corresponding to "Beyoncé"

inputs = tokenizer(text, entity_spans=entity_spans, return_tensors="pt")
inputs.to("cuda")
outputs = model(**inputs)

predicted_indices = [index for index, logit in enumerate(outputs.logits[0]) if logit > 0]
print("Predicted entity type for Beyoncé:", [model.config.id2label[index] for index in predicted_indices])

entity_spans = [(17, 28)]  # character-based entity span corresponding to "Beyoncé"
inputs = tokenizer(text, entity_spans=entity_spans, return_tensors="pt")
inputs.to("cuda")
outputs = model(**inputs)

predicted_indices = [index for index, logit in enumerate(outputs.logits[0]) if logit > 0]
print("Predicted entity type for Los Angeles:", [model.config.id2label[index] for index in predicted_indices])

Predicted entity type for Beyoncé: ['person']
Predicted entity type for Los Angeles: ['location', 'place']
